# Machine Learning in a Nutshell: Binary Classifier

## A (given) Binary Classifier

To begin, we assume the weights of a binary classifier are given as $w_1 = 1.0$ and $w_2 = 0.5$.
In the second task we are going to learn the weights from data.

We also assume, the feature extractor $\phi(\mathbf{x})$ is given as the identity function. For simplicity, we are working with the extracted feature vectors $[x_1, x_2]$ directly.

### Our classifier

$$
\begin{align}
f_\mathbf{w}(x) &= \text{sign}(\underbrace{[1, 0.5]}_{\mathbf{w}} \cdot \underbrace{[x_1, x_2]}_{\phi(x)})\\
      &= \text{sign}(\underbrace{1}_{w_1} \cdot x_1 + \underbrace{0.5}_{w_2} \cdot x_2)
\end{align}
$$

### The sign-function maps from a score to a label
$$
\begin{equation*}
  \text{sign}(z)=\begin{cases}
    +1, & \text{if $z>0$}\\
    -1, & \text{if $z<0$}\\
     0, & \text{if $z = 0$}.
  \end{cases}
\end{equation*}
$$

### Task 1: Making predictions with the classifier $f$

In order to make predictions, we need to implement $f$ in Python. This requires us to implement the individual pieces as python methods:
- A `getScore` method that computes the **score**: $w_1 \cdot x_1 + w_2 \cdot x_2$
- A `getLabel` method that computes the **label** from a given **score**: $\text{sign}(\cdot)$. Implement the `sign` function yourself!
- A `predict` method that computes the **label** from two parameters $x_1$ and $x_2$ by calling the other methods.

Define a class and implement the `getScore`, `getLabel` and `predict` methods.
The constructor should expect two parameters $w_0$ and $w_1$ and store the values in instance variables.

In [12]:
class BinaryClassifier:
    def __init__(self, w1, w2) -> None:
        self.w1 = w1
        self.w2 = w2

    def getScore(self, x1, x2):
        pass

    def getLabel(self, score):
        pass

    def predict(self, x1, x2):
        pass

    def verbosePrediction(self, x1, x2, y):
        pass

In [13]:
# implement a method called predict that takes two arguments x1 and x2

w1 = 15.0
w2 = 4.5

x1 = 2
x2 = -2

classifier = BinaryClassifier(w1, w2)
print(classifier.predict(x1, x2))

1


### Task 2: Multiple predictions and the target labels

Below you are given a list of training examples.
Each training example comes with a target label.

The target label is the label our classifier should predict, ... but most likely it doesn't since we did not train it yet.

Let's first make the classifier predict a label for each training example and print the predicted label next to the target label such we can see when things go wrong.

#### Implement printing as a method of `BinaryClassifier`, named `verbosePrediction` that expects $x_1$, $x_2$ and $y$ as parameters and prints, for each training example, the predicted label and the target label and if it is correct.

In [14]:
data = [(0.5, 0.5), (2, 0), (-1, 1), (1, -1), (1, -2), (-1, -1)]
labels = [1, 1, 1, -1, -1, -1]

for x, y in zip(data, labels):
    classifier.verbosePrediction(x[0], x[1], y)


x1: 0.5 x2:0.5 -> 1, label 1, correct?True
x1: 2 x2:0 -> 1, label 1, correct?True
x1: -1 x2:1 -> -1, label 1, correct?False
x1: 1 x2:-1 -> 1, label -1, correct?False
x1: 1 x2:-2 -> 1, label -1, correct?False
x1: -1 x2:-1 -> -1, label -1, correct?True


### Task 3: Compute the margin

Recall, the **score** of a training example is positive if the angle between the feature vector of the training example and the weight vector is acute. And the score is negative if the angle is obtuse.
$$
\mathbf{w} \cdot \mathbf{x} = \Vert \mathbf{w} \Vert \cdot \Vert \mathbf{x} \Vert \cdot \cos(\omega)
$$

All points in feature space with a score = 0, constitute the decision boundary of the classifier.

Now, the **margin** takes the score and the target label and quantifies the correctness of a prediction:
$$
\text{margin}(\mathbf{x}, y, \mathbf{w}) = (w_1 \cdot x_1 + w_2 \cdot x_2) \cdot y
$$

The margin is positive if the score and the target label agree (regardless of which class it is). It is negative, if both quantities disagree.

Add a method `getMargin` to your implementation and, additionally, print the margin for each training example in the `predictVerbose` method.


### Task 4: Compute the Hinge-Loss of the classifier

Based on the margin, we can now calculate the Hinge-Loss of the classifier.

Like any other loss, the Hinge Loss is large if the classifier's prediction does **not** agree with the target label.
This is the exact opposite of what the margin tells us.

Thus, we can just use the negative margin to indicate disagreement between prediction and target label.

$$
\text{Loss}_{hinge}(x, y, \mathbf{w}) = \max \{ \text{gap} - \underbrace{\underbrace{(\mathbf{w} \cdot \mathbf{x})}_{\text{score}}y}_{\text{margin}}, 0 \}
$$

Intuitively, what is the role of $\text{gap}$?
- What if $gap = 0$?
- What if $gap > 0$?

Hint: Think about the closeness of training points to the decision boundary (Or rather the closeness of the decision boundary to the training points, since it's up to us to adjust the decision boundary based on the loss).
![](linear-classifier-decision-boundary.png)

Implement a method `getHingeLoss` that expects parameters $x_1$, $x_2$ and, additionally, a keyword parameter `gap` with a default value of $1.0$.

### Task 5: Implement the weight update

Based on the hinge loss, we can construct a weight update rule and repeatedly update the weights.

The goal of repeated weight updates is to adjust the decision boundary of the classifier, such that it classifies the training examples correctly.

For each training example, we calculate the weight change $\Delta \mathbf{w}$ and then update the weights using $\Delta \mathbf{w}$. The update rule is given as:

$$
\mathbf{w} = \mathbf{w} - 0.1 \cdot \Delta \mathbf{w}
$$

where the weight change is given by:

$$
\begin{align}
\Delta \mathbf{w}  = \begin{cases}
    - [x_1 \cdot y, x_2 \cdot y], & \qquad \text{if} \quad \text{Loss}_{hinge}(\mathbf{x}, y, \mathbf{w}) > 0\\
    0, & \qquad \text{otherwise}.
  \end{cases}
\end{align}
$$

Note: We obtain the weight change by calculating the partial derivatives of the loss with respect to each weight. Example for $w_1$:
$$
\begin{align}
\frac{\partial \text{Loss}_{hinge}(\mathbf{x}, y, \mathbf{w})}{\partial w_1} &= \frac{\partial}{\partial w_1} (\text{gap} - (w_1 \cdot x_1 + w_2 \cdot x_2) \cdot y)\\
   &= \frac{\partial}{\partial w_1} \text{gap} - \frac{\partial}{\partial w_1} (w_1 \cdot x_1 + w_2 \cdot x_2) \cdot y)\\
   &= 0 - \frac{\partial}{\partial w_1} (w_1 \cdot x_1 + w_2 \cdot x_2) \cdot \frac{\partial}{\partial (w_1 \cdot x_1 + w_2 \cdot x_2)} (w_1 \cdot x_1 + w_2 \cdot x_2) \cdot y \\
   &= 0 - \frac{\partial}{\partial w_1} (w_1 \cdot x_1 \cdot + w_2 \cdot x_2)  \cdot y\\
   &= 0 - \frac{\partial}{\partial w_1} (w_1 \cdot x_1 \cdot y)  \cdot y\\
   &= - x_1 \cdot y\\
\end{align}
$$

The derivates for $w_2$ are computed analogously.
The weight change $\Delta \mathbf{w}$ is just the vector of these partial derivatives.

#### Implement the update rule in a method `train`. This method expects $x_1$, $x_2$ and $y$ as parameters. You may also include an optional `eta` parameter for the learning rate, which is currently set to a fixed 0.1.

### Task 6: Train the classifier until it predicts all training points correctly

Steps:
- Loop over a number of epochs (e.g. 100)
    - Initialize a variable `trainLoss = 0`
    - Loop over all training examples
        - Call `train` on each example
        - Call `getHingeLoss` on each example and add the loss to `trainLoss`
    - Print the average `trainLoss`
- Verify that your classifier now makes correct predictions using the method `printStats`

In [ ]:
classifier = BinaryClassifier(w1, w2)
n_epochs = 60
eta = 0.05

for epoch in range(n_epochs):
    trainLoss = 0.
    # TODO: iterate over training data here
    # Calculate the loss (sum up and average/epoch) using the getHingeLoss function
    # Train the classifier using your .train method 
    # Check if your predictions are correct by printing results



### Congratulations! You have implemented the core machinery of machine learning algorithms.